In [1]:
import os

In [2]:
import pandas

In [3]:
from pyiron import Project

In [4]:
pr = Project("pyiron")

In [5]:
pr.remove_jobs_silently(recursive=True)

In [6]:
structure = pr.create_ase_bulk("Cu")

In [7]:
job = pr.create_job(pr.job_type.Lammps, "lmp_runner")

In [8]:
job.structure = structure

In [9]:
job.potential = 'Cu-runner'

In [10]:
job.potential

,Config,Filename,Model,Name,Species,Citations
0,"[pair_style nnp dir ""."" showew no showewsum 0 resetew no maxew 100 cflength 1.8897261328 cfenergy 0.0367493254 emap ""1:Cu""\n, pair_coeff * * 12\n]","[runner/input.nn, runner/weights.029.data, runner/scaling.data]",RuNNer,Cu-runner,[Cu],{}


In [11]:
job.run()

The job lmp_runner was saved and received the ID: 1


In [12]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,1,finished,Cu,lmp_runner,/lmp_runner,/home/jovyan/,pyiron/,2021-03-07 14:46:51.920669,2021-03-07 14:46:52.477082,0.0,pyiron@jupyter-jan-2djanssen-2dpotential-5ftesting-2dzql63ef2#1,Lammps,0.1,None,None
